In [1]:
import pandas as pd
import numpy as np
import torch

C:\Users\vinee\miniconda3\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('bigBasketProducts.csv')
data.head()
data = data.drop('rating', axis=1)
# Drop all rows with null values in a particular column
column_with_nulls = 'description'
data = data.dropna(subset=[column_with_nulls])
data = data.dropna(subset=['brand'])
data = data.dropna(subset=['product'])

In [3]:
len(np.unique(data['category'][:]))

11

In [4]:
data['Concatenated'] = data.apply(lambda row: '|'.join([f'{col}:{val}' for col, val in row.items() if col != 'index']), axis=1)
data['Concatenated'][0]

'product:Garlic Oil - Vegetarian Capsule 500 mg|category:Beauty & Hygiene|sub_category:Hair Care|brand:Sri Sri Ayurveda |sale_price:220.0|market_price:220.0|type:Hair Oil & Serum|description:This Product contains Garlic Oil that is known to help proper digestion, maintain proper cholesterol levels, support cardiovascular and also build immunity.  For Beauty tips, tricks & more visit https://bigbasket.blog/'

In [5]:
import torch
from transformers import BertTokenizer, BertModel

# Assuming 'df' is your DataFrame
# You can replace this with the actual name of your DataFrame

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [6]:
sentences = data['Concatenated'].tolist()
from tqdm import tqdm
# Example function to process a batch
def process_batch(batch):
    # Replace this with your actual function logic
    print(f"Processing batch: {batch}")

# Split the list into batches of 100
batch_size = 32
batched_sentences = [sentences[i:i+batch_size] for i in range(0, len(sentences), batch_size)]
embeddings = []
# Iterate over batches and call the process_batch function
for sentences in tqdm(batched_sentences):
    # Tokenize input sentences
    tokenized_input = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True)

    # Move the input tensors to GPU
    tokenized_input = {key: value.to(device) for key, value in tokenized_input.items()}

    # Forward pass through BERT model
    with torch.no_grad():
        output = model(**tokenized_input)
    del(tokenized_input)
    # Extract embeddings from the output
    embedding = output.last_hidden_state.mean(dim=1)  # Using mean pooling to get sentence embeddings
    embeddings.append(embedding.cpu().detach())
    del(embedding)
# print(sentence_embeddings.size())

100%|████████████████████████████████████████████████████████████████████████████████| 858/858 [27:40<00:00,  1.94s/it]


In [11]:
embeddings_tensor = torch.cat(embeddings, dim=0)

In [13]:
embeddings_tensor.size()

torch.Size([27439, 768])

In [14]:
torch.save(embeddings,'embeddings.pt')

In [16]:
embeddings = embeddings_tensor.numpy()

In [33]:
embeddings = embeddings.astype(np.float32)

In [63]:
import pinecone      

# pinecone.init(      
# 	api_key='ec5d5e0d-f392-4654-ad4c-d030f2f8adc7',      
# 	environment='us-east1-gcp'      
# )      
# index = pinecone.Index('bigbasket')

import pinecone      

pinecone.init(      
	api_key='1b7c40a9-2a02-4f0c-b096-2e5d34ce2366',      
	environment='gcp-starter'      
)      
index = pinecone.Index('bigbasket')

In [64]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [53]:
upload = []
for i in range(len(embeddings)):
    e = embeddings[i].tolist()
    id = str(i)
    tup = (id,e)
    upload.append(tup)

In [ ]:
# Alternatively, upload vectors in batches
batch_size = 100
for i in tqdm(range(0, len(upload), batch_size)):
    batch_vectors = upload[i:i + batch_size]
    response = index.upsert(batch_vectors)
    # print(f"Inserted batch with IDs {batch_ids[0]} to {batch_ids[-1]}, Pinecone response: {response}")

 15%|████████████▎                                                                    | 42/275 [01:51<10:22,  2.67s/it]